**Задача №5**

Для решения использован следующий алгоритм:

    Начнем с того, чтобы разжечь костер и осмотреться вокруг.
    Создадим матрицу, где каждая клетка будет представлять собой либо стену, либо пустое место.
    Используя информацию из пункта 1, обновим соответствующие клетки в матрице.
    Если есть доступные клетки для перемещения, направим минотавра в одну из них. Если перед минотавром стена, необходимо повернуть его в нужном направлении.
    Повторяем шаги 3-4, пока не обойдем все клетки лабиринта.
    В конце выведем общее время, затраченное на обход.

Для упрощения реализации, я буду использовать декартову систему координат, где направления будут иметь следующие значения:

    Вверх: (0, 1)
    Вниз: (0, -1)
    Влево: (-1, 0)
    Вправо: (1, 0)

In [ ]:
class Minotaur:
    def __init__(self, x, y, x1, y1, A, B, C, K):
        self.x = x
        self.y = y
        self.direction = (x1 - x, y1 - y)
        self.A = A
        self.B = B
        self.C = C
        self.K = K
        self.time_spent = 0
        self.maze = {}
        self.visited = set()

    def turn(self, clockwise=True):
        if self.direction == (0, 1):  # Up
            self.direction = (1, 0) if clockwise else (-1, 0)
        elif self.direction == (1, 0):  # Right
            self.direction = (0, -1) if clockwise else (0, 1)
        elif self.direction == (0, -1):  # Down
            self.direction = (-1, 0) if clockwise else (1, 0)
        elif self.direction == (-1, 0):  # Left
            self.direction = (0, 1) if clockwise else (0, -1)
        self.time_spent += self.B
        return 2, 1 if clockwise else 0

    def move(self):
        new_x = self.x + self.direction[0]
        new_y = self.y + self.direction[1]
        if (new_x, new_y) not in self.maze or self.maze[(new_x, new_y)] == "#":
            self.time_spent += self.A
            return 1, 0
        else:
            self.x, self.y = new_x, new_y
            self.time_spent += self.A
            return 1, 1

    def light_fire(self):
        self.time_spent += self.C
        return 3

    def update_maze(self, view):
        for i, row in enumerate(view):
            for j, cell in enumerate(row):
                x_coord = self.x + j - self.K
                y_coord = self.y + i - self.K
                self.maze[(x_coord, y_coord)] = cell

    def all_visited(self):
        for cell, value in self.maze.items():
            if value == "_" and cell not in self.visited:
                return False
        return True

    def choose_next_move(self):
        for d in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x = self.x + d[0]
            new_y = self.y + d[1]
            if (new_x, new_y) in self.maze and self.maze[(new_x, new_y)] == "_" and (new_x, new_y) not in self.visited:
                while self.direction != d:
                    self.turn()
                return self.move()
        return self.light_fire()


class MazeSimulator:
    def __init__(self, maze):
        self.maze = maze
        self.minotaur_pos = None

    def get_response(self, action, *additional_data):
        if action == 3:  # Light fire
            view = []
            for i in range(-additional_data[0], additional_data[0] + 1):
                row = []
                for j in range(-additional_data[0], additional_data[0] + 1):
                    x, y = self.minotaur_pos[0] + j, self.minotaur_pos[1] + i
                    if 0 <= x < len(self.maze[0]) and 0 <= y < len(self.maze):
                        row.append(self.maze[y][x])
                    else:
                        row.append("#")
                view.append("".join(row))
            return view
        elif action == 1:  # Move
            dx, dy = additional_data
            new_x = self.minotaur_pos[0] + dx
            new_y = self.minotaur_pos[1] + dy
            if self.maze[new_y][new_x] == "#":
                return 0
            else:
                self.minotaur_pos = (new_x, new_y)
                return 1
        elif action == 2:  # Turn
            return 1

    def set_minotaur_position(self, x, y):
        self.minotaur_pos = (x, y)


def navigate_maze_with_simulation(initial_data, maze):
    minotaur = Minotaur(*initial_data)
    simulator = MazeSimulator(maze)
    simulator.set_minotaur_position(initial_data[0], initial_data[1])
    actions = []

    # Start by lighting the fire to view the surroundings
    action = minotaur.light_fire()
    actions.append(action)
    response = simulator.get_response(action, minotaur.K)
    minotaur.update_maze(response)

    while not minotaur.all_visited():
        action, *additional_data = minotaur.choose_next_move()
        actions.append((action, *additional_data))

        if action == 1:  # For move action, add direction to additional_data
            additional_data = [minotaur.direction[0], minotaur.direction[1]]

        response = simulator.get_response(action, *additional_data)
        if action == 1 and response == 1:
            minotaur.visited.add((minotaur.x, minotaur.y))
        elif action == 3:
            minotaur.update_maze(response)

    # Add the terminal command
    actions.append((4, minotaur.time_spent))

    return actions


# Test the function using a sample maze
maze_sample = [
    "#######",
    "#_____#",
    "#_###_#",
    "#_#_#_#",
    "#_#_#_#",
    "#_____#",
    "#######"
]
initial_data_sample = (1, 1, 2, 1, 3, 1, 1, 1)
actions_sample = navigate_maze_with_simulation(initial_data_sample, maze_sample)
print(actions_sample)